## Demo

Алгоритм працює в декілька проходів.

Групи, у яких коефіціент варіації < 20% (задається параметром), вважаються стабільними.

Спочатку намагаємось формувати стабільні групи по 2 мікросервіси. Далі, із тих, що залишились, пробуємо формувати стабільні групи по 3 елементи. І так далі, збільшуємо кількість елементів в групі, допоки не закінчаться елементи, або не досягнемо максимальної кількості елементів (параметр, по дефолту = 4). 


In [ ]:
def group_original_microservices(microservices, available_indices, max_group_size, stability_threshold, 
                                final_groups, final_group_indices, final_slot_sums):
    """
    Групує оригінальні мікросервіси, перебираючи різні розміри груп
    
    Args:
        microservices: Список часових рядів з навантаженням мікросервісів
        available_indices: Список доступних індексів мікросервісів
        max_group_size: Максимальна кількість елементів у групі
        stability_threshold: Поріг для коефіцієнта варіації
        final_groups: Список груп для доповнення
        final_group_indices: Список індексів мікросервісів у кожній групі
        final_slot_sums: Список загальних навантажень за часовими слотами для кожної групи
        
    Returns:
        Оновлений список доступних індексів
    """
    n = len(microservices)
    
    # Ітеруємо за розмірами груп від 2 до max_group_size
    for group_size in range(2, min(max_group_size + 1, n + 1)):
        print(f"\nПробуємо розмір групи: {group_size}")
        
        if len(available_indices) < group_size:
            print(f"  Недостатньо сервісів залишилось для розміру групи {group_size}")
            break
        
        # Генеруємо стабільні групи для поточного розміру
        candidate_groups = generate_stable_groups(
            available_indices, 
            microservices, 
            group_size, 
            stability_threshold
        )
        
        if not candidate_groups:
            print(f"  Не знайдено стабільних груп розміром {group_size}")
            continue
        
        print(f"  Знайдено {len(candidate_groups)} стабільних груп розміром {group_size}")
        
        # Створюємо набір всіх вже використаних індексів для швидкого пошуку
        used_indices_set = set(idx for group in final_group_indices for idx in group)
        
        # Додаємо стабільні групи до фінальних груп
        for group, actual_indices, cv in candidate_groups:
            # Пропускаємо, якщо будь-який індекс вже використовується
            if not is_group_available(actual_indices, used_indices_set):
                continue
                
            # Додаємо цю групу до фінальних груп
            final_groups.append(group)
            final_group_indices.append(actual_indices)
            final_slot_sums.append(calculate_load_sum(group))
            
            # Оновлюємо набір використаних індексів
            used_indices_set.update(actual_indices)
            
            print(f"  Додано групу з CV: {cv:.2f}% - {actual_indices}")
        
        # Оновлюємо доступні індекси, видаляючи використані
        available_indices = [idx for idx in available_indices if idx not in used_indices_set]
        
        print(f"  Залишилось сервісів: {len(available_indices)}")
        
        # Якщо не залишилось мікросервісів, завершуємо
        if not available_indices:
            break
    
    return available_indices

Якщо залишаються мікросервіси, які не потрапили в жодну групу. Обрізаємо їм пікові значення.

In [13]:
def process_unassigned_microservices(unassigned_indices, microservices):
    """
    Обробляє мікросервіси, які не вдалося згрупувати, 
    розділяючи їх на базові та пікові компоненти.
    
    Args:
        unassigned_indices: Індекси мікросервісів, які не вдалося згрупувати
        microservices: Список усіх мікросервісів
    
    Returns:
        Кортеж з двох списків: 
        (списки базових компонентів з індексами, списки пікових компонентів з індексами)
    """
    base_services = []
    base_indices = []
    peak_services = []
    peak_indices = []
    
    for idx in unassigned_indices:
        microservice = microservices[idx]
        base_load, peak_load = split_microservice_load(microservice)
        
        # Додаємо базовий компонент зі збереженням індексу
        base_services.append(base_load)
        base_indices.append(idx)
        
        # Додаємо піковий компонент, лише якщо він містить ненульові значення
        if any(x > 0 for x in peak_load):
            peak_services.append(peak_load)
            peak_indices.append(idx)
    
    return (base_services, base_indices), (peak_services, peak_indices) 

In [ ]:
def split_microservice_load(time_series, threshold_std=0.1):
    """
    Розділяє навантаження мікросервісу на базову та пікову складову.
    
    Алгоритм:
    1. Знаходить пікові значення, які перевищують середнє + threshold_std * стандартне_відхилення
    2. Розширює область піків, включаючи суміжні значення, що перевищують середнє
    3. Розділяє часовий ряд на дві компоненти:
       - базову (обмежену максимальним непіковим значенням)
       - пікову (надлишкове навантаження)
    
    Args:
        time_series: Часовий ряд навантаження мікросервісу
        threshold_std: Поріг у стандартних відхиленнях для визначення піків (за замовчуванням 0.1)
    
    Returns:
        Кортеж з двох списків: (базове_навантаження, пікове_навантаження)
    """
    # Перетворення в numpy array, якщо це не array
    time_series = np.array(time_series)
    
    # Обчислюємо середнє та стандартне відхилення
    mean = np.mean(time_series)
    std_dev = np.std(time_series)
    
    # Визначаємо екстремуми (аномалії)
    extremes = time_series[(time_series > mean + threshold_std * std_dev)]
    
    # Якщо екстремумів немає, повертаємо оригінальний ряд та нульовий ряд
    if len(extremes) == 0:
        return time_series.tolist(), [0] * len(time_series)
    
    # Знаходимо індекси екстремумів
    extreme_indices = np.where(time_series > mean + threshold_std * std_dev)[0]
    
    # Знаходимо індекси першого та останнього екстремуму
    first_idx = extreme_indices[0]
    last_idx = extreme_indices[-1]
    
    # Розширюємо область екстремумів ліворуч
    extremes_extension_left = []
    extreme_indices_left = []
    for i in range(first_idx - 1, -1, -1):
        if time_series[i] > mean:
            extremes_extension_left.append(time_series[i])
            extreme_indices_left.append(i)
        else:
            break
    
    # Розширюємо область екстремумів праворуч
    extremes_extension_right = []
    extreme_indices_right = []
    for i in range(last_idx + 1, len(time_series)):
        if time_series[i] > mean:
            extremes_extension_right.append(time_series[i])
            extreme_indices_right.append(i)
        else:
            break
    
    # Об'єднуємо всі індекси екстремумів та розширення
    all_extreme_indices = sorted(extreme_indices_left + extreme_indices.tolist() + extreme_indices_right)
    
    # Створюємо маску для значень, які не входять в розширену область екстремумів
    mask = np.ones(len(time_series), dtype=bool)
    mask[all_extreme_indices] = False
    
    # Знаходимо максимальне значення за виключенням екстремумів
    # Це буде порогом для "базового" навантаження
    if np.any(mask):
        new_max_value = np.max(time_series[mask])
    else:
        # Якщо всі значення є екстремумами, використовуємо середнє значення
        new_max_value = mean
    
    # Створюємо базовий ряд (обмежений новим максимумом)
    base_load = np.minimum(time_series, new_max_value)
    
    # Створюємо піковий ряд (надлишок над новим максимумом)
    peak_load = np.maximum(time_series - new_max_value, 0)
    
    return base_load.tolist(), peak_load.tolist()

Далі мікросервіси з обрізаними піками проганяються на повторний пошук

In [ ]:
def group_base_components(base_services, base_indices, base_available, max_group_size, stability_threshold,
                         temp_groups, temp_indices, temp_slots):
    """
    Групує базові компоненти
    
    Args:
        base_services: Список часових рядів базових компонентів
        base_indices: Список індексів вихідних мікросервісів
        base_available: Список доступних індексів базових компонентів
        max_group_size: Максимальна кількість елементів у групі
        stability_threshold: Поріг для коефіцієнта варіації
        temp_groups: Список тимчасових груп для доповнення
        temp_indices: Список тимчасових індексів для доповнення
        temp_slots: Список тимчасових сум навантажень для доповнення
        
    Returns:
        Оновлений список доступних індексів базових компонентів
    """
    # Повторюємо цикл для базових компонентів
    for group_size in range(2, min(max_group_size + 1, len(base_services) + 1)):
        print(f"\nПробуємо розмір групи для базових компонентів: {group_size}")
        
        if len(base_available) < group_size:
            print(f"  Недостатньо базових компонентів для розміру групи {group_size}")
            break
        
        # Генеруємо стабільні групи базових компонентів
        candidate_groups = generate_stable_groups(
            base_available, 
            base_services, 
            group_size, 
            stability_threshold
        )
        
        if not candidate_groups:
            print(f"  Не знайдено стабільних груп розміром {group_size}")
            continue
        
        print(f"  Знайдено {len(candidate_groups)} стабільних груп розміром {group_size}")
        
        # Створюємо набір всіх вже використаних індексів для швидкого пошуку
        used_base_set = set(idx for group in temp_indices for idx in group)
        
        # Додаємо стабільні групи до тимчасових груп
        for group, actual_indices, cv in candidate_groups:
            # Пропускаємо, якщо будь-який індекс вже використовується
            if not is_group_available(actual_indices, used_base_set):
                continue
                
            # Додаємо цю групу до тимчасових груп
            temp_groups.append(group)
            temp_indices.append(actual_indices)
            temp_slots.append(calculate_load_sum(group))
            
            # Оновлюємо набір використаних індексів
            used_base_set.update(actual_indices)
            
            # Виводимо індекси оригінальних мікросервісів для зрозумілості
            real_indices = [base_indices[idx] for idx in actual_indices]
            print(f"  Додано групу базових компонентів з CV: {cv:.2f}% - оригінальні індекси: {real_indices}")
        
        # Оновлюємо доступні базові компоненти
        base_available = [idx for idx in base_available if idx not in used_base_set]
        
        print(f"  Залишилось базових компонентів: {len(base_available)}")
        
        # Якщо не залишилось базових компонентів, завершуємо
        if not base_available:
            break
    
    return base_availabledef group_base_components(base_services, base_indices, base_available, max_group_size, stability_threshold,
                         temp_groups, temp_indices, temp_slots):
    """
    Групує базові компоненти
    
    Args:
        base_services: Список часових рядів базових компонентів
        base_indices: Список індексів вихідних мікросервісів
        base_available: Список доступних індексів базових компонентів
        max_group_size: Максимальна кількість елементів у групі
        stability_threshold: Поріг для коефіцієнта варіації
        temp_groups: Список тимчасових груп для доповнення
        temp_indices: Список тимчасових індексів для доповнення
        temp_slots: Список тимчасових сум навантажень для доповнення
        
    Returns:
        Оновлений список доступних індексів базових компонентів
    """
    # Повторюємо цикл для базових компонентів
    for group_size in range(2, min(max_group_size + 1, len(base_services) + 1)):
        print(f"\nПробуємо розмір групи для базових компонентів: {group_size}")
        
        if len(base_available) < group_size:
            print(f"  Недостатньо базових компонентів для розміру групи {group_size}")
            break
        
        # Генеруємо стабільні групи базових компонентів
        candidate_groups = generate_stable_groups(
            base_available, 
            base_services, 
            group_size, 
            stability_threshold
        )
        
        if not candidate_groups:
            print(f"  Не знайдено стабільних груп розміром {group_size}")
            continue
        
        print(f"  Знайдено {len(candidate_groups)} стабільних груп розміром {group_size}")
        
        # Створюємо набір всіх вже використаних індексів для швидкого пошуку
        used_base_set = set(idx for group in temp_indices for idx in group)
        
        # Додаємо стабільні групи до тимчасових груп
        for group, actual_indices, cv in candidate_groups:
            # Пропускаємо, якщо будь-який індекс вже використовується
            if not is_group_available(actual_indices, used_base_set):
                continue
                
            # Додаємо цю групу до тимчасових груп
            temp_groups.append(group)
            temp_indices.append(actual_indices)
            temp_slots.append(calculate_load_sum(group))
            
            # Оновлюємо набір використаних індексів
            used_base_set.update(actual_indices)
            
            # Виводимо індекси оригінальних мікросервісів для зрозумілості
            real_indices = [base_indices[idx] for idx in actual_indices]
            print(f"  Додано групу базових компонентів з CV: {cv:.2f}% - оригінальні індекси: {real_indices}")
        
        # Оновлюємо доступні базові компоненти
        base_available = [idx for idx in base_available if idx not in used_base_set]
        
        print(f"  Залишилось базових компонентів: {len(base_available)}")
        
        # Якщо не залишилось базових компонентів, завершуємо
        if not base_available:
            break
    
    return base_available

Пікові компоненти виносяться в кінці в окремі групи

In [ ]:
def finalize_results(final_groups, final_group_indices, final_slot_sums,
                   peak_components, peak_indices, peak_slot_sums):
    """
    Формує фінальні результати, додаючи пікові компоненти в кінці
    
    Args:
        final_groups: Список груп
        final_group_indices: Список індексів мікросервісів у кожній групі
        final_slot_sums: Список загальних навантажень за часовими слотами для кожної групи
        peak_components: Список пікових компонентів
        peak_indices: Список індексів пікових компонентів
        peak_slot_sums: Список сум навантажень пікових компонентів
        
    Returns:
        Кортеж з (groups, group_services, slot_sums)
    """
    # Додаємо пікові компоненти в самому кінці
    if peak_components:
        print("\nДодаємо пікові компоненти як окремі групи в самому кінці:")
        for i, (group, indices, slots) in enumerate(zip(peak_components, peak_indices, peak_slot_sums)):
            final_groups.append(group)
            final_group_indices.append(indices)
            final_slot_sums.append(slots)
            print(f"  Додано піковий компонент мікросервісу: {-indices[0]}")
    
    return final_groups, final_group_indices, final_slot_sums

## Імпорт

In [10]:
import main
import json

## базовий приклад з 6 тайм слотами

In [8]:
print("\n" + "="*60)
print("Input data:")
    
microservices = [
    [1, 2, 3, 4, 3, 2],  # Microservice 0
    [2, 1, 1, 2, 3, 4],  # Microservice 1
    [3, 3, 2, 1, 2, 3],  # Microservice 2
    [4, 3, 2, 1, 1, 2],  # Microservice 3
    [2, 3, 4, 3, 2, 1],  # Microservice 4
    [1, 2, 3, 4, 4, 3],  # Microservice 5
    [3, 2, 1, 2, 3, 4],  # Microservice 6
    [4, 3, 2, 1, 2, 3],  # Microservice 7
]
    
for i, service in enumerate(microservices):
    print(f"Microservice {i}: {service}")

print("="*60 + "\n")

try:
    groups, group_services, slot_sums = main.form_multiple_knapsack_groups(microservices)
    main.print_results(groups, group_services, slot_sums)
except ValueError as e:
    print(f"Error: {e}")
        


Input data:
Microservice 0: [1, 2, 3, 4, 3, 2]
Microservice 1: [2, 1, 1, 2, 3, 4]
Microservice 2: [3, 3, 2, 1, 2, 3]
Microservice 3: [4, 3, 2, 1, 1, 2]
Microservice 4: [2, 3, 4, 3, 2, 1]
Microservice 5: [1, 2, 3, 4, 4, 3]
Microservice 6: [3, 2, 1, 2, 3, 4]
Microservice 7: [4, 3, 2, 1, 2, 3]


Trying with group size: 2
  Found 9 stable groups with size 2
  Added group with CV: 0.00% - [0, 7]
  Added group with CV: 0.00% - [3, 5]
  Added group with CV: 0.00% - [4, 6]
  Remaining services: 2

Trying with group size: 3
  Not enough services left for group size 3
  Added single-element group: 1
  Added single-element group: 2
Grouping results:

Group 1:
  Microservices: [0, 7]
  Individual load by time slots:
    Microservice 0: [1, 2, 3, 4, 3, 2]
    Microservice 7: [4, 3, 2, 1, 2, 3]
  Total load by time slots: [5, 5, 5, 5, 5, 5]
  Average load: 5.00
  Standard deviation: 0.00
  Coefficient of variation: 0.00%

Group 2:
  Microservices: [3, 5]
  Individual load by time slots:
    Microse

## приклад з 24 тайм слотами

(тут генерував дані мікросервісів аі-шкою)

In [9]:
with open('testing/realistic_data.json', 'r') as f:
    real_data = json.load(f)
    
print(f"Завантажено {len(real_data)} мікросервісів з реальними патернами навантаження")

# Запуск алгоритму групування
groups, group_services, slot_sums = main.form_multiple_knapsack_groups(real_data)
print(f"Алгоритм створив {len(groups)} груп")
main.print_results(groups, group_services, slot_sums)

# Аналіз стабільності груп
group_stats = []
for i, (group, services, sums) in enumerate(zip(groups, group_services, slot_sums)):
    mean = sum(sums) / len(sums)
    variance = sum((x - mean) ** 2 for x in sums) / len(sums)
    std_dev = variance ** 0.5
    cv = (std_dev / mean) * 100 if mean > 0 else float('inf')
    
    group_stats.append({
        'group_id': i,
        'size': len(services),
        'mean': mean,
        'std_dev': std_dev,
        'cv': cv,
        'services': services
    })

# Сортування груп за стабільністю (CV)
group_stats.sort(key=lambda x: x['cv'])

# Вивід статистики
print("\nСтатистика груп (відсортована за коефіцієнтом варіації):")
print(f"{'ID групи':<10} {'Розмір':<10} {'Сер. навант.':<15} {'Станд. відх.':<15} {'Коеф. варіації':<15}")
print("-" * 65)

for stat in group_stats[:10]:  # Виводимо тільки 10 найкращих груп
    print(f"{stat['group_id']:<10} {stat['size']:<10} {stat['mean']:.2f}{' '*11} {stat['std_dev']:.2f}{' '*11} {stat['cv']:.2f}%")


Завантажено 50 мікросервісів з реальними патернами навантаження

Trying with group size: 2
  Found 378 stable groups with size 2
  Added group with CV: 0.00% - [13, 47]
  Added group with CV: 0.00% - [16, 44]
  Added group with CV: 0.00% - [18, 48]
  Added group with CV: 0.00% - [30, 42]
  Added group with CV: 0.00% - [31, 40]
  Added group with CV: 0.00% - [36, 49]
  Added group with CV: 0.00% - [9, 43]
  Added group with CV: 0.00% - [8, 46]
  Added group with CV: 0.00% - [7, 41]
  Added group with CV: 0.00% - [38, 45]
  Added group with CV: 5.03% - [3, 4]
  Added group with CV: 5.52% - [0, 2]
  Added group with CV: 6.57% - [1, 6]
  Added group with CV: 14.52% - [15, 25]
  Added group with CV: 14.70% - [17, 27]
  Added group with CV: 15.28% - [23, 26]
  Added group with CV: 15.52% - [10, 34]
  Added group with CV: 15.65% - [22, 28]
  Added group with CV: 15.89% - [20, 32]
  Added group with CV: 15.92% - [11, 33]
  Added group with CV: 16.16% - [21, 29]
  Remaining services: 8

Trying 